In [1]:
import torch

In [2]:
a = torch.ones(2,2)
b = torch.rand_like(a)
print(a)
print(b)

tensor([[1., 1.],
        [1., 1.]])
tensor([[0.6797, 0.4401],
        [0.8853, 0.5934]])


---

# 使用方法

执行运算并反向传播

In [3]:
a.requires_grad = True
c = a @ b
print(c)

tensor([[1.5650, 1.0335],
        [1.5650, 1.0335]], grad_fn=<MmBackward0>)


In [4]:
c.backward(torch.ones_like(c))

查看张量`a`的导数

In [5]:
print(a.grad)

tensor([[1.1198, 1.4787],
        [1.1198, 1.4787]])


`autograd`的功能已经整合进Tensor类当中了

使用自动求导功能只需要在建立Tensor时设定`requires_grad=True`或后续修改

在进行`backword`时会自动计算并存入`tensor.grad`当中

---

`autograd`支持tensor之间的标量运算，也支持矩阵乘法

---

# 自定义函数

如果我们需要自定义一个函数，让这个函数作为自动求导的一部分

我们需要继承`torch.autograd.function.Function`这个类

重写`Function`类的`forward`和`backword`这两个静态方法

In [6]:
from typing import Any, override
from torch.autograd.function import Function

class MulScaler(Function):
    @override
    @staticmethod
    def forward(ctx, tensor:torch.Tensor, scaler:float) -> torch.Tensor:
        ctx.scaler = scaler
        return tensor * scaler
    
    @override
    @staticmethod
    def backward(ctx, grad_output) -> torch.Tensor:
        return grad_output * ctx.scaler, None

In [7]:
a = torch.ones(4,4, requires_grad=True)
a_2 = MulScaler.apply(a, 4)
a_2.backward(torch.ones_like(a_2))
print('a_2:\n', a_2)
print('a.grad:\n', a.grad)

a_2:
 tensor([[4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.]], grad_fn=<MulScalerBackward>)
a.grad:
 tensor([[4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.],
        [4., 4., 4., 4.]])


通过自定义的函数`MalScaler`实现了`autograd`的正向传播和反向求导

---